In [ ]:
def find_backdoor_sets(G, nodes):
    """
    G: causal graph
    nodes: the nodes in casaul graph
    """
    backdoor_sets = defaultdict(list)
    for node in nodes:
        for adj_node in G.predecessors(node):
            backdoor_sets[node].append(adj_node)
    return backdoor_sets

In [ ]:
def adjust_for_backdoor(df, var, intervention_value, bd_set, target_column):
    # Adjust a single variable for backdoor paths
    adjusted_values = []
    for y in df[target_column].unique():
        adjusted_prob = backdoor_adjustment(df, target_column, y, var, intervention_value, bd_set)
        adjusted_values.append(adjusted_prob)
    return adjusted_values

def ranking_query(G, df, k, update_vars, target_column, condition=None):
    df_temp = df.copy()
    # ... [Existing filtering code]

    backdoor_sets = find_backdoor_sets(G, G.nodes())

    for var, value in update_vars.items():
        if var in backdoor_sets:
            bd_set = backdoor_sets[var]
            adjusted_values = adjust_for_backdoor(df_temp, var, value, bd_set, target_column)
            # Apply the adjusted probabilities to the target column
            df_temp[target_column] = adjusted_values

    # ... [Existing ranking code]

    return df_temp.loc[top_k_indices]